In [1]:
# Step 2: Import pandas
import pandas as pd
from IPython.display import display

In [2]:
# Step 3: Load the Excel file
df = pd.read_excel("Book11.xlsx")

# Step 4: Display the DataFrame
df.head()

,Car Number,SOC Initial,Preference Slots,Route,Priority,Battery Capacity,Full Charge Range,KM before CS,Required time(If user provide),SOC Final,Required Time (Calculated)
0,23123,35,1,1,0,90.0,350.0,25.0,15.0,30,16
1,657,6,1,3,0,NaN,NaN,NaN,13.0,3,12
2,45345,28,1,1,1,NaN,NaN,NaN,15.0,22,13
3,768678,44,1,1,2,NaN,NaN,NaN,NaN,40,14
4,6756,12,1,1,3,NaN,NaN,NaN,NaN,8,15


In [3]:
# Step 3: Load the Excel file
df2 = pd.read_excel("CS2_freetime.xlsx")
cs2_Free_Slot_duration = df2['Vacancy Duration'].tolist()
# Create a new DataFrame to store deleted rows
CS2_vehicles = pd.DataFrame()


In [4]:
Preference_Slots = df['Preference Slots'].unique().tolist()
Preference_Slots.sort()
print(Preference_Slots)

[1, 2, 3]


In [5]:
for i,j in zip(Preference_Slots,cs2_Free_Slot_duration):
    # Select the rows where 'Preference Slots' is 1
    df_preference_1 = df[df['Preference Slots'] == i]
    # Drop rows where 'SOC Final' is below 10
    df_preference_1_filtered = df_preference_1[df_preference_1['SOC Final'] >= 10]
    # Reorder the rows according to the values of 'priority' & then 'SOC Final' in descending order
    df_preference_1_filtered_sorted = df_preference_1_filtered.sort_values(by=['Priority','SOC Final'])
    # Calculate the sum of the 'Required Time (Calculated)' column
    total_required_time = df_preference_1_filtered_sorted['Required Time (Calculated)'].sum()

    # Initialize X with the calculated sum
    X = total_required_time

    # Traverse from below to top and delete rows where 'Route' is 2 until X < 180
    data_reversed = df_preference_1_filtered_sorted.iloc[::-1]
    if X > 180 :
        for index, row in data_reversed.iterrows():
            if row['Route'] == 2 and row['SOC Final'] >=30 :
                X -= row['Required Time (Calculated)']
                CS2_vehicles = CS2_vehicles.append(row)
                df_preference_1_filtered_sorted.drop(index, inplace=True)
                if X < 180:
                    break
    print(X)            
    if X > 180 :
        for index, row in data_reversed.iterrows():
            if row['Route'] == 1 :
                X -= row['Required Time (Calculated)']
                df_preference_1_filtered_sorted.drop(index, inplace=True)
                if X < 180:
                    break
    print(X)
    # Display the updated DataFrame and the final value of X
    updated_data = df_preference_1_filtered_sorted.reset_index(drop=True)
    display(updated_data)
    CS2_vehicles = CS2_vehicles.reset_index(drop=True)
    # Calculate the sum of the 'Required Time (Calculated)' column for charging station 2
    Y = CS2_vehicles['Required Time (Calculated)'].sum()
    print(Y)
    if Y >=j :
        while Y >= j:
            min_index = CS2_vehicles['SOC Final'].idxmax()
            Y -= CS2_vehicles.loc[min_index, 'Required Time (Calculated)']
            CS2_vehicles.drop(min_index, inplace=True)
    print(Y) 
    display(CS2_vehicles)
    # Drop all rows
    CS2_vehicles = CS2_vehicles.iloc[0:0]

    # Export each DataFrame to a separate CSV file
    file_name = f'slot_{i}_updated_cs1.csv'
    updated_data.to_csv(file_name, index=False)
    print(f'DataFrame {i+1} exported to {file_name}')

206.0
169.0


C:\Users\USER\AppData\Local\Temp\ipykernel_19328\3437879913.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  CS2_vehicles = CS2_vehicles.append(row)
C:\Users\USER\AppData\Local\Temp\ipykernel_19328\3437879913.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  CS2_vehicles = CS2_vehicles.append(row)
C:\Users\USER\AppData\Local\Temp\ipykernel_19328\3437879913.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  CS2_vehicles = CS2_vehicles.append(row)


,Car Number,SOC Initial,Preference Slots,Route,Priority,Battery Capacity,Full Charge Range,KM before CS,Required time(If user provide),SOC Final,Required Time (Calculated)
0,23123,35,1,1,0,90.0,350.0,25.0,15.0,30,16
1,24,35,1,1,0,NaN,NaN,NaN,17.0,31,15
2,45345,28,1,1,1,NaN,NaN,NaN,15.0,22,13
3,76786,65,1,1,1,NaN,NaN,NaN,NaN,60,18
4,5677,28,1,2,2,NaN,NaN,NaN,15.0,24,17
5,768678,44,1,1,2,NaN,NaN,NaN,NaN,40,14
6,75675,14,1,2,3,NaN,NaN,NaN,NaN,10,16
7,452,16,1,3,3,NaN,NaN,NaN,NaN,12,16
8,5645,18,1,3,3,NaN,NaN,NaN,NaN,15,14
9,7967,31,1,3,3,NaN,NaN,NaN,NaN,27,13


47.0
32.0


,Car Number,SOC Initial,Preference Slots,Route,Priority,Battery Capacity,Full Charge Range,KM before CS,Required time(If user provide),SOC Final,Required Time (Calculated)
1,7867.0,46.0,1.0,2.0,3.0,NaN,NaN,NaN,16.0,40.0,17.0
2,79879.0,41.0,1.0,2.0,3.0,NaN,NaN,NaN,NaN,35.0,15.0


...

C:\Users\USER\AppData\Local\Temp\ipykernel_19328\3437879913.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  CS2_vehicles = CS2_vehicles.append(row)
C:\Users\USER\AppData\Local\Temp\ipykernel_19328\3437879913.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  CS2_vehicles = CS2_vehicles.append(row)
C:\Users\USER\AppData\Local\Temp\ipykernel_19328\3437879913.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  CS2_vehicles = CS2_vehicles.append(row)


,Car Number,SOC Initial,Preference Slots,Route,Priority,Battery Capacity,Full Charge Range,KM before CS,Required time(If user provide),SOC Final,Required Time (Calculated)
0,2312334,35,3,1,0,90.0,350.0,25.0,15.0,30,16
1,1110900,35,3,1,0,NaN,NaN,NaN,17.0,31,15
2,4539121,28,3,1,1,NaN,NaN,NaN,15.0,22,13
3,76531113,65,3,1,1,NaN,NaN,NaN,NaN,60,18
4,4343009,28,3,2,2,NaN,NaN,NaN,15.0,24,17
5,1118666,44,3,1,2,NaN,NaN,NaN,NaN,40,14
6,7569922,14,3,2,3,NaN,NaN,NaN,NaN,10,16
7,12102909,16,3,3,3,NaN,NaN,NaN,NaN,12,16
8,5645099,18,3,3,3,NaN,NaN,NaN,NaN,15,14
9,7998971,31,3,3,3,NaN,NaN,NaN,NaN,27,13


49.0
15.0


,Car Number,SOC Initial,Preference Slots,Route,Priority,Battery Capacity,Full Charge Range,KM before CS,Required time(If user provide),SOC Final,Required Time (Calculated)
2,7981211.0,41.0,3.0,2.0,3.0,NaN,NaN,NaN,NaN,35.0,15.0


DataFrame 4 exported to slot_3_updated_cs1.csv
